In [2]:
import fastjet as fj
import time
import matplotlib.pyplot as plt
import numpy as np
import pyhepmc

In [32]:
def readFile(number):
    datafile = open(f"./total/{number}.txt","r")
    lines = datafile.readlines()
    datafile.close()

    return lines

def loadParticles(lines):
    event = []

    for line in lines:
        arr = line.split("\t")
        particle = fj.PseudoJet(float(arr[0]), float(arr[1]), float(arr[2]), float(arr[3]))

        event.append(particle)
    
    return event

def makeJets(data, R):
    jet_def = fj.JetDefinition(fj.cambridge_aachen_algorithm, R)
    jet_cluster = fj.ClusterSequence(data, jet_def)
    # jets = jet_cluster.inclusive_jets()

    # return jets
    return jet_cluster

In [33]:
ptmin = 2.0

file = readFile(0)
particle_data = loadParticles(file)
start = time.time()
cluster = makeJets(particle_data, 1.0)
end = time.time() - start
jets = cluster.inclusive_jets(ptmin)

print(end)

0.0007138252258300781


### Using Mass Drop Tagger

In [34]:
mu = 0.67
ycut = 0.09
mass_drop_tagger = fj.MassDropTagger(mu, ycut)

print(len(jets))
n = 0

count = 0

tagged = []

for n in range(len(jets)):
    start = time.time()

    tagged_jet = mass_drop_tagger.result(jets[n])
    
    print(time.time() - start)
    print(n, f"\tOriginal jet: pt={jets[n].pt()}, eta={jets[n].eta()}, phi={jets[n].phi()}, mass={jets[n].m()}")
    print(n, f"\tTagged jet: pt={tagged_jet.pt()}, eta={tagged_jet.eta()}, phi={tagged_jet.phi()}, mass={tagged_jet.m()}\n")
    
    tagged.append(tagged_jet)
    
final_jets = fj.sorted_by_pt([mass_drop_tagger.result(jet) for jet in jets])

print("------------------------------------")
print(len(final_jets))

with open("results/massdrop-python.csv", "w") as FILE:
    FILE.write("pt,eta,m\n")
    for jet in final_jets:
        if jet.pt() > 0:
            print(jet.pt(), jet.eta(), jet.m(), sep="\t")
            FILE.write(f"{jet.pt()},{jet.eta()},{jet.m()}\n")

print(count)

11
1.52587890625e-05
0 	Original jet: pt=4.25496424823028, eta=0.9892011098399707, phi=2.4143391152495934, mass=2.522418674135956
0 	Tagged jet: pt=3.5649614011641453, eta=1.0286210400408577, phi=2.5256138204970515, mass=1.385375128391125

1.0013580322265625e-05
1 	Original jet: pt=2.2464885078239303, eta=1.7455644977280314, phi=3.6063896578231613, mass=1.8375069953643175
1 	Tagged jet: pt=2.2464885078239303, eta=1.7455644977280314, phi=3.6063896578231613, mass=1.8375069953643175

5.7220458984375e-06
2 	Original jet: pt=2.48768233091831, eta=-0.8177598891625226, phi=0.1329137506706785, mass=1.3796583711669916
2 	Tagged jet: pt=2.48768233091831, eta=-0.8177598891625226, phi=0.1329137506706785, mass=1.3796583711669916

6.198883056640625e-06
3 	Original jet: pt=3.3439531295491505, eta=-1.9436688109037505, phi=1.5352538003514755, mass=3.300867237447968
3 	Tagged jet: pt=3.3439531295491505, eta=-1.9436688109037505, phi=1.5352538003514755, mass=3.300867237447968

5.0067901611328125e-06
4 	Or

### Using Filter

In [9]:
fil = fj.Filter(fj.JetDefinition(fj.cambridge_algorithm, 0.3), fj.SelectorNHardest(3))
final_jets = fj.sorted_by_pt([fil(jet) for jet in jets])

print("------------------------------------")
print(len(final_jets))

with open("results/filter-python.csv", "w") as FILE:
    FILE.write("pt,eta,m\n")
    for jet in final_jets:
        print(jet.pt(), jet.eta(), jet.m(), sep="\t")
        FILE.write(f"{jet.pt()},{jet.eta()},{jet.m()}\n")


------------------------------------
11
25.425973541479426	3.3899727842078757	7.210829212029252
19.38426838643532	-2.2264724312433675	4.940888635018613
12.019482608698311	1.7377047604888647	6.671551849243314
4.118477623409108	0.981058208612941	2.3690413802846306
4.070582559375079	-0.22492429567807123	2.3285876831086623
2.487682330918309	-0.8177598891625233	1.3796583711669919
2.1864737162176358	-1.8293104858994278	2.1299068355434225
2.125720970693987	2.418204608888062	1.8245962501869402
1.986731603795744	-8.097398676211913	2.234241203271326
1.7682801418035814	-1.5580514503303071	1.742592726752077
1.6337885215517716	1.786135408697485	0.8683760196849447


### Using Trimming

In [16]:
fil = fj.Filter(fj.JetDefinition(fj.cambridge_algorithm, 0.3), fj.SelectorPtFractionMin(0.3))
final_jets = fj.sorted_by_pt([fil(jet) for jet in jets])

print("------------------------------------")
print(len(final_jets))

with open("results/trim-python.csv", "w") as FILE:
    FILE.write("pt,eta,m\n")
    for jet in final_jets:
        if jet.pt() > 0:
            print(jet.pt(), jet.eta(), jet.m(), sep="\t")
            FILE.write(f"{jet.pt()},{jet.eta()},{jet.m()}\n")

------------------------------------
11
24.0024657948891	3.39693136347249	5.931458819456554
11.39058332934317	-2.1796071349581148	1.3116812665798905
8.958194272123187	1.4009830801346865	2.466264379925911
3.5649614011641453	1.0286210400408577	1.385375128391125
2.1376462300800347	-0.7410704326313319	1.172218172070111
2.125720970693987	2.418204608888062	1.8245962501869402
1.8254297278011031	-0.03236046511747068	0.9395700000000008
1.6925532105156107	-8.110365603402714	2.0683933326506128
1.539181615169957	-1.5893036495193658	0.935056154256617


### Benchmarking the different modules

In [27]:
def read_hepmc_file(FILE):
    total = []
    with pyhepmc.open(f"./benchmarks/data/{FILE}", "r") as f:
        
        for i, event in enumerate(f):
            particles = []
            for j, particle in enumerate(event.particles):
                if particle.status == 1: 
                # Access particle properties such as momentum, energy, etc.
                # print(particle.momentum.px)
                    px, py, pz = particle.momentum.px, particle.momentum.py, particle.momentum.pz
                    energy = particle.momentum.e

                    particles.append(fj.PseudoJet(px, py, pz, energy))
            total.append(particles)

    return total

fileName = ["events-ee-Z.hepmc3", "events-ee-H.hepmc3", \
                "events-pp-0.5TeV-5GeV.hepmc3", "events-pp-1TeV-5GeV.hepmc3",\
                "events-pp-2TeV-5GeV.hepmc3", "events-pp-1TeV.hepmc3", "events-pp-2TeV.hepmc3",\
                "events-pp-5TeV-10GeV.hepmc3", "events-pp-8TeV-20GeV.hepmc3",\
                "events-pp-13TeV-20GeV.hepmc", "events-pp-20TeV-20GeV.hepmc3", "events-pp-20TeV-50GeV.hepmc3",\
                "events-pp-30TeV-50GeV.hepmc3"]

trials = 15

with open("./results/time-massdrop-python.csv", "w") as FILE:
    FILE.write("FileName,mean_particles,n_samples,time_per_event\n")
    for eventFILE in fileName:
        allEvents = read_hepmc_file(eventFILE)

        avgTime = 0.0
        for j in range(trials):
            avgEvent = 0.0
            count = 0
            
            for i in range(100):
                count += len(allEvents[i])
                cluster = makeJets(allEvents[i], 1.0)
                jets = cluster.inclusive_jets(2.0)
                mu = 0.67
                ycut = 0.09
                mass_drop_tagger = fj.MassDropTagger(mu, ycut)
                start = time.time()
                final_jets = [mass_drop_tagger(jet) for jet in jets]
                
                end = time.time() - start
                print(final_jets[0])
                avgEvent += end

            
            avgTime += avgEvent

            # count += len(particle_data)
        avgTime /= trials

        print(f"{eventFILE}\t{count/100}\t{trials}\t{avgTime*10**4}\n")
        FILE.write(f"{eventFILE},{count/100},{trials},{avgTime*10**4}\n")


events-ee-Z.hepmc3	43.05	15	7.648626963297526

events-ee-H.hepmc3	64.97	15	8.23227564493815

events-pp-0.5TeV-5GeV.hepmc3	112.62	15	14.360427856445312

events-pp-1TeV-5GeV.hepmc3	160.36	15	20.509084065755207

events-pp-2TeV-5GeV.hepmc3	188.21	15	31.053225199381508

events-pp-1TeV.hepmc3	226.98	15	26.3980229695638

events-pp-2TeV.hepmc3	226.98	15	26.748021443684895

events-pp-5TeV-10GeV.hepmc3	284.15	15	36.50585810343424

events-pp-8TeV-20GeV.hepmc3	354.18	15	40.83728790283203

events-pp-13TeV-20GeV.hepmc	431.18	15	49.218972524007164

events-pp-20TeV-20GeV.hepmc3	524.59	15	57.62751897176107

events-pp-20TeV-50GeV.hepmc3	553.64	15	56.17173512776693

events-pp-30TeV-50GeV.hepmc3	632.29	15	64.25110499064127

